# Pokus z ineho repa, pre SSD quant...
model:

https://github.com/qfgaohao/pytorch-ssd

dataset:

http://host.robots.ox.ac.uk/pascal/VOC/voc2007/

cite:

@misc{pascal-voc-2007,
	author = "Everingham, M. and Van~Gool, L. and Williams, C. K. I. and Winn, J. and Zisserman, A.",
	title = "The {PASCAL} {V}isual {O}bject {C}lasses {C}hallenge 2007 {(VOC2007)} {R}esults",
	howpublished = "http://www.pascal-network.org/challenges/VOC/voc2007/workshop/index.html"}


In [0]:
from vision.ssd import mobilenet_v2_ssd_lite

model = mobilenet_v2_ssd_lite

In [1]:
! python eval_ssd.py --net mb2-ssd-lite  --dataset /home/bohumil/FIIT/BP/BP/Zdroje_kod/VOCtest_06-Nov-2007/VOCdevkit/VOC2007 --trained_model models/mb2-ssd-lite-mp-0_686.pth --label_file models/voc-model-labels.txt 


eval_ssd.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  all_gt_boxes[class_index][image_id] = torch.tensor(all_gt_boxes[class_index][image_id])
It took 0.08451247215270996 seconds to load the model.
process image 0
Load Image: 0.014594 seconds.
Inference time:  0.08942389488220215
Prediction: 0.130992 seconds.
process image 1
Load Image: 0.005113 seconds.
Inference time:  0.01656937599182129
Prediction: 0.218346 seconds.
process image 2
Load Image: 0.013486 seconds.
Inference time:  0.01397848129272461
Prediction: 0.110415 seconds.
process image 3
Load Image: 0.009514 seconds.
Inference time:  0.013193368911743164
Prediction: 0.121053 seconds.
process image 4
Load Image: 0.011102 seconds.
Inference time:  0.011215686798095703
Prediction: 0.091208 seconds.
process image 5
Load Image: 0.010527 seconds.
Inference time:  0.01117753982

# Base precision

Average Precision Per-class:
aeroplane: 0.6986083268890578
bicycle: 0.7785683817533883
bird: 0.6378586823413062
boat: 0.5504846489536213
bottle: 0.35583246357894943
bus: 0.7903270853943654
car: 0.7420740183930007
cat: 0.8245088852819729
chair: 0.5400334049922988
cow: 0.6190354978742831
diningtable: 0.7301573841470218
dog: 0.7850487846790698
horse: 0.8231384239695627
motorbike: 0.8161471728488682
person: 0.7161793135440326
pottedplant: 0.42674861706699757
sheep: 0.6264582479587281
sofa: 0.7864885994228247
train: 0.8351225228790526
tvmonitor: 0.6579004592801784

Average Precision Across All Classes:0.6870360460624291

## Quantization - prepare model

In [ ]:
net = create_mobilenetv2_ssd_lite(len(class_names), width_mult=args.mb2_width_mult, is_test=True)
timer.start("Load Model")
net.load(args.trained_model)
net = net.to(DEVICE)

predictor = create_mobilenetv2_ssd_lite_predictor(net, nms_method=args.nms_method, device=DEVICE)

In [1]:
import logging
def config_notebooks_logger():
    logging.config.fileConfig('logging.conf')
    msglogger = logging.getLogger()
    msglogger.info('Logging configured successfully')
    return msglogger

In [2]:
import argparse
import distiller

msglogger = config_notebooks_logger()

parser = argparse.ArgumentParser()
distiller.quantization.add_post_train_quant_args(parser)
args = parser.parse_args(args= [])
# args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet18_imagenet_post_train.yaml'

Logging configured successfully


In [8]:
from eval_ssd_quant import make_quantized, eval
import torch

# --net mb2-ssd-lite  --dataset /home/bohumil/FIIT/BP/BP/Zdroje_kod/VOCtest_06-Nov-2007/VOCdevkit/VOC2007 
# --trained_model models/mb2-ssd-lite-mp-0_686.pth --label_file models/voc-model-labels.txt
ssd_parser = argparse.ArgumentParser()
ssd_args = ssd_parser.parse_args(args=[])

ssd_args.net = 'mb2-ssd-lite'
ssd_args.dataset = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/VOCtest_06-Nov-2007/VOCdevkit/VOC2007'
ssd_args.trained_model = 'models/mb2-ssd-lite-mp-0_686.pth'
ssd_args.label_file = 'models/voc-model-labels.txt'
ssd_args.eval_dir = 'eval_results'
ssd_args.mb2_width_mult = 1.0
ssd_args.use_cuda = False
# ssd_args.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() and ssd_args.use_cuda else "cpu")
ssd_args.DEVICE = "cpu"
ssd_args.nms_method="hard"
ssd_args.dataset_type = 'voc'
ssd_args.use_2007_metric=True
ssd_args.iou_threshold=0.5

model, predictor= make_quantized(ssd_args)

It took 24.324840545654297 seconds to load the model.


pydev debugger: warning: trying to add breakpoint to file that does not exist: /SSD_attempt.ipynb (will have no effect)


In [15]:
eval(model, predictor, ssd_args, num_pictures=200)

No labels file, using default VOC classes.


process image 0


RuntimeError: expected device cpu but got device cuda:0

Correct way of getting statistics

In [5]:
cpu_model = distiller.make_non_parallel_copy(model)

TypeError: can't pickle module objects

In [6]:
model.cpu()

SSD(
  (base_net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Co

In [8]:
def eval_model(model, ssd_args):
    ssd_args.use_cuda = False
    ssd_args.DEVICE = "cpu"
    eval(model, predictor, ssd_args, num_pictures=200)
    

In [9]:
from distiller.data_loggers import collect_quant_stats, QuantCalibrationStatsCollector, collector_context


args.qe_calibration = 0.2
if args.qe_calibration:
    
    cpu_model = model.cpu()
    
    distiller.utils.assign_layer_fq_names(cpu_model)
    msglogger.info("Generating quantization calibration stats based on {0} users".format(args.qe_calibration))
    collector = distiller.data_loggers.QuantCalibrationStatsCollector(cpu_model)
    with collector_context(collector):
        eval_model(cpu_model, ssd_args)
        # Here call your model evaluation function, making sure to execute only
        # the portion of the dataset specified by the qe_calibration argument
    yaml_path = './act_quantization_stats.yaml'
    collector.save(yaml_path)

Generating quantization calibration stats based on 0.2 users
No labels file, using default VOC classes.


process image 0


RuntimeError: expected device cpu but got device cuda:0

In [ ]:
from copy import deepcopy
def eval_quantized(model, args):
    if args.quantize_eval:
        quantizer = distiller.quantization.PostTrainLinearQuantizer.from_args(deepcopy(model), args)
        # dummy = distiller.get_dummy_input(model.input_shape)
        dummy = distiller.get_dummy_input(input_shape=model.input_shape)
        quantizer.prepare_model(dummy)
        eval_model(dataloader, quantizer.model, 'cuda', print_freq=30)

In [ ]:
args.qe_config_file = '/home/bohumil/FIIT/BP/BP/Zdroje_kod/quantization_jupyters/resnet18_imagenet_post_train.yaml'
eval_quantized(model, args)

